In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
import pickle as pickle

In [5]:
df = pd.read_csv(r'archive\tripadvisor_hotel_reviews.csv')
print(df.head())

                                              Review  Rating
0  nice hotel expensive parking got good deal sta...       4
1  ok nothing special charge diamond member hilto...       2
2  nice rooms not 4* experience hotel monaco seat...       3
3  unique, great stay, wonderful time hotel monac...       5
4  great stay great stay, went seahawk game aweso...       5


In [6]:
df = df[['Review', 'Rating']]
df['sentiment'] = df['Rating'].apply(lambda x: 'positive' if x > 3 
                                    else 'negative' if x < 3
                                    else 'neutral')
df = df[['Review', 'sentiment']]
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['Review'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['Review'])
padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post')

In [8]:
sentiment_labels = pd.get_dummies(df['sentiment']).values

In [9]:
# Creating a train-test split
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, sentiment_labels, test_size=0.2)

In [10]:
# Adding layers to the model
model = Sequential()
model.add(Embedding(5000, 100, input_length=100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          500000    
                                                                 
 conv1d (Conv1D)             (None, 96, 64)            32064     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                        

In [11]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
513/513 [==============================] - 8s 14ms/step - loss: 0.5805 - accuracy: 0.7864 - val_loss: 0.4399 - val_accuracy: 0.8351
Epoch 2/10
513/513 [==============================] - 7s 14ms/step - loss: 0.3768 - accuracy: 0.8564 - val_loss: 0.4068 - val_accuracy: 0.8463
Epoch 3/10
513/513 [==============================] - 7s 13ms/step - loss: 0.2639 - accuracy: 0.8957 - val_loss: 0.4189 - val_accuracy: 0.8365
Epoch 4/10
513/513 [==============================] - 7s 13ms/step - loss: 0.1805 - accuracy: 0.9323 - val_loss: 0.5510 - val_accuracy: 0.8456
Epoch 5/10
513/513 [==============================] - 7s 13ms/step - loss: 0.1223 - accuracy: 0.9562 - val_loss: 0.5800 - val_accuracy: 0.8219
Epoch 6/10
513/513 [==============================] - 7s 13ms/step - loss: 0.0844 - accuracy: 0.9725 - val_loss: 0.6692 - val_accuracy: 0.8231
Epoch 7/10
513/513 [==============================] - 7s 13ms/step - loss: 0.0537 - accuracy: 0.9824 - val_loss: 0.9250 - val_accuracy: 0.8346

In [12]:
y_pred = np.argmax(model.predict(x_test), axis=-1)
print("Accuracy:", accuracy_score(np.argmax(y_test, axis=-1), y_pred))

129/129 [==============================] - 0s 3ms/step
Accuracy: 0.8297145645279337


## Saving the model

In [13]:
model.save('sentiment_analysis_model.h5')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Loading the Model

In [2]:
import keras
import pickle

model = keras.models.load_model('sentiment_analysis_model.h5')
with open('tokenizer.pickle','rb') as handle:
    tokenizer = pickle.load(handle)

In [6]:
def predict_sentiment(text):
    # Tokenize and pad the input text
    text_sequence = tokenizer.texts_to_sequences([text])
    text_sequence = pad_sequences(text_sequence, maxlen=100)

    # Make a prediction using the trained model
    predicted_rating = model.predict(text_sequence)[0]
    if np.argmax(predicted_rating) == 0:
        return 'Negative'
    elif np.argmax(predicted_rating) == 1:
        return 'Neutral'
    else:
        return 'Positive'

In [11]:
text_input = "It's an best place"
predicted_sentiment = predict_sentiment(text_input)
print(predicted_sentiment)

1/1 [==============================] - 0s 17ms/step
Positive
